In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt

# Program creates dict with 3 DataFrames: 

In [2]:
def create_dfs(*args):
    d_frames = {}
    for year in args:
        file_path = '{}.xlsx'.format(year)
        d_frames[year] = pd.read_excel(file_path)
        d_frames[year]['Year'] = year
    return d_frames

In [3]:
d_frames = create_dfs(2014,2015,2016)

In [4]:
d_frames[2014].head(2)

,Country_ISO,City_name,Station_EoI_Code,Station_name,TypeOfMeasurementStation,AreaOfMeasurementStation,AirPollutant,AirPollutionLevel (µg/m3),Latitude,Longitude,Altitude,Year
0,AT,Graz,AT60171,Graz Ost Petersgasse,Traffic,urban,PM10,51.038625,47.059444,15.466125,362,2014
1,AT,Linz,AT4S431,Linz Römerberg B139,Traffic,urban,PM10,48.000000,48.303055,14.283070,262,2014


In [5]:
d_frames[2015].head(2)

,Country,City,AirQualityStationEoICode,AQStationName,AirPollutant,AirPollutionLevel,UnitOfAirpollutionLevel,AirQualityStationType,AirQualityStationArea,Longitude,Latitude,Altitude,Year
0,Albania,NaN,AL0205A,Durres,PM10,33.502958,µg/m3,Traffic,urban,19.44920,41.3199,1.0,2015
1,Albania,NaN,AL0206A,Shkoder,PM10,33.462417,µg/m3,Background,urban,19.52342,42.3139,13.0,2015


In [6]:
d_frames[2016].head(2)

,Country,City,AirQualityStationEoICode,AQStationName,AirPollutant,AirPollutionLevel,UnitOfAirpollutionLevel,AirQualityStationType,AirQualityStationArea,Longitude,Latitude,Altitude,Year
0,Montenegro,NaN,ME0001A,Pljevlja,PM10,228.280250,ug/m3,Background,urban,19.358101,43.355034,777.0,2016
1,Turkey,NaN,TR760141,IGDIR,PM10,220.916667,ug/m3,Background,urban,44.053610,39.926110,859.0,2016


# Extracting columns of intrest, and unifing column names:

In [7]:
d_frames[2014] = d_frames[2014][['Station_EoI_Code','AreaOfMeasurementStation','AirPollutionLevel (µg/m3)','Year','Latitude','Longitude']]
d_frames[2015] = d_frames[2015][['AirQualityStationEoICode','AirQualityStationArea','AirPollutionLevel','Year','Latitude','Longitude']]
d_frames[2016] = d_frames[2016][['AirQualityStationEoICode','AirQualityStationArea','AirPollutionLevel','Year','Latitude','Longitude']]

In [8]:
d_frames[2014].columns = d_frames[2016].columns
d_frames[2015].columns = d_frames[2016].columns

## Concatinating all 3 DF from dictionary with pd.concat. With unified columns names no extra argument needs to be pased:

In [9]:
pm10_data = pd.concat(d_frames, sort=False, )
pm10_data.head()

AirQualityStationEoICode AirQualityStationArea  AirPollutionLevel  \
2014 0                  AT60171                 urban          51.038625   
     1                  AT4S431                 urban          48.000000   
     2                  AT60164                 urban          47.800000   
     3                  AT4S415                 urban          46.700375   
     4                  AT90TAB                 urban          46.154000   

        Year   Latitude  Longitude  
2014 0  2014  47.059444  15.466125  
     1  2014  48.303055  14.283070  
     2  2014  47.055553  15.417789  
     3  2014  48.323334  14.297792  
     4  2014  48.218056  16.381958

## Extracting country abbrevation with accesing string atribiutes of one of the column. 

In [10]:
pm10_data['Country']=pm10_data['AirQualityStationEoICode'].str[:2]

## Setting Indexes

In [11]:
pm10_data = pm10_data.set_index(['Year'])

In [12]:
pm10_data.head()

,AirQualityStationEoICode,AirQualityStationArea,AirPollutionLevel,Latitude,Longitude,Country
Year,,,,,,
2014,AT60171,urban,51.038625,47.059444,15.466125,AT
2014,AT4S431,urban,48.000000,48.303055,14.283070,AT
2014,AT60164,urban,47.800000,47.055553,15.417789,AT
2014,AT4S415,urban,46.700375,48.323334,14.297792,AT
2014,AT90TAB,urban,46.154000,48.218056,16.381958,AT


In [13]:
pm10_data[pm10_data['AirQualityStationEoICode'] == 'IT1334A']

,AirQualityStationEoICode,AirQualityStationArea,AirPollutionLevel,Latitude,Longitude,Country
Year,,,,,,
2014,IT1334A,urban,23.000000,46.495277,11.340278,IT
2015,IT1334A,urban,28.958333,46.495280,11.340280,IT
2016,IT1334A,urban,31.708333,46.495280,11.340280,IT


# Creating interactive map with folium library. Red markers indicates exceed of EU daily limit value of particulate matter < 10 μm(PM10).


In [14]:
def locations_on_map(year):
    pm10_map = folium.Map(location=[46.49,11.34], zoom_start=4)
    df = pm10_data.loc[year]
    for idx, row in df.iterrows():
        col="blue"
        if row['AirPollutionLevel'] > 50:
            col = "red"
        marker = folium.Marker(location=[row['Latitude'],row['Longitude']], 
                               popup= row['AirQualityStationEoICode'], tooltip= row['AirPollutionLevel'], 
                               icon=folium.Icon(color = col))
        marker.add_to(pm10_map)

    return pm10_map

In [15]:
locations_on_map(2014)